<a href="https://colab.research.google.com/github/Srvand/NLP_Transformers/blob/main/BertModel_Updated_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.6 MB/s 
     |████████████████████████████████| 3.3 MB 28.7 MB/s 
     |████████████████████████████████| 596 kB 43.1 MB/s 
     |████████████████████████████████| 895 kB 43.9 MB/s 
     |████████████████████████████████| 61 kB 480 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
# Load Huggingface transformers
from transformers import TFBertModel,  BertConfig, BertTokenizerFast,BertTokenizer
# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.utils import to_categorical
# And pandas for data import + sklearn because you allways need sklearn
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
# Import data from csv
df = pd.read_csv('data.csv').drop(['Unnamed: 0'],axis=1)



In [9]:
df.head(5)

,text,category,category_label
0,I am still waiting on my card?,12,card_arrival
1,What can I do if my card still hasn't arrived ...,12,card_arrival
2,I have been waiting over a week. Is the card s...,12,card_arrival
3,Can I track my card while it is in the process...,12,card_arrival
4,"How do I know if I will get my card, or if it ...",12,card_arrival


In [5]:
df['category_label']=pd.Categorical(df['category'])
df['category'] = df['category_label'].cat.codes

In [6]:
# Split into train and test
train,test = train_test_split(df, test_size = 0.2, stratify = df['category'])

In [10]:
print(train.shape,test.shape)

(10466, 3) (2617, 3)


In [11]:
X_train,X_test,Y_train,Y_test = train_test_split(df.text.values, df.category.values, test_size=0.2)

In [12]:
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

(10466,) (2617,) (10466,) (2617,)


In [16]:
Y_train

array([21, 54, 29, ..., 67, 66, 39], dtype=int8)

In [17]:
# Name of the BERT model to use
model_name = 'bert-base-uncased'
# Max length of tokens
max_length = 100

# Load transformers config and set output_hidden_states to False
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
# Load the Transformers BERT model
transformer_model = TFBertModel.from_pretrained(model_name, config = config)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [18]:
config

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [ ]:
# x = tokenizer(
#     text=train['text'].to_list(),
#     add_special_tokens=True,
#     max_length=max_length,
#     truncation=True,
#     padding=True, 
#     return_tensors='tf',
#     return_token_type_ids = False,
#     return_attention_mask = False,
#     verbose = True)
  

In [19]:
def batch_encode(X, tokenizer):
    return tokenizer.batch_encode_plus(
    X,
    max_length=max_length, # set the length of the sequences
    # truncation=True,
    add_special_tokens=True, # add [CLS] and [SEP] tokens
    return_attention_mask=False,
    return_token_type_ids=False, # not needed for this type of ML task
    pad_to_max_length=True, # add 0 pad tokens to the sequences less than max_length
    return_tensors='tf'
)

In [ ]:
batch_encode(X_train, tokenizer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


{'input_ids': <tf.Tensor: shape=(10466, 100), dtype=int32, numpy=
array([[  101,  1996,  3863, ...,     0,     0,     0],
       [  101,  2045,  2003, ...,     0,     0,     0],
       [  101,  2054, 27218, ...,     0,     0,     0],
       ...,
       [  101,  1045,  2215, ...,     0,     0,     0],
       [  101,  2054,  5491, ...,     0,     0,     0],
       [  101,  2026,  2327, ...,     0,     0,     0]], dtype=int32)>}

In [20]:
# Load the MainLayer
bert = transformer_model.layers[0]
# Build your model input
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
# input_ids = Input(name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}
# Load the Transformers BERT model as a layer in a Keras model
bert_model = bert(inputs)[1]
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(bert_model, training=False)
# Then build your model output
category = Dense(units=len(train.category_label.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='category')(pooled_output)
outputs = {'category': category}
# And combine it all in a model object
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')

In [21]:
bert_model

<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'bert')>

In [22]:
model.summary()

Model: "BERT_MultiLabel_MultiClass"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, 100)]             0         
                                                                 
 bert (TFBertMainLayer)      TFBaseModelOutputWithPoo  109482240 
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             100, 768),                          
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                        

In [ ]:
Y_train

array([11, 66, 64, ..., 41, 13, 10], dtype=int8)

In [23]:
### ------- Train the model ------- ###
# Set an optimizer
optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
# Set loss and metrics
loss = {'category': SparseCategoricalCrossentropy(from_logits = True)}
metric = {'category': SparseCategoricalAccuracy('accuracy')}
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)
# Ready output data for the model
y_category = Y_train

# Tokenize the input (takes some time)
x = batch_encode(X_train,tokenizer)
# Fit the model
history = model.fit(
    x={'input_ids': x['input_ids']},
    y={'category': y_category},
    validation_split=0.2,
    batch_size=64,
    epochs=1)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


131/131 [==============================] - 337s 2s/step - loss: 4.2682 - accuracy: 0.0260 - val_loss: 3.9814 - val_accuracy: 0.0712


In [36]:
test_y_category =Y_test
test_x =batch_encode(X_test,tokenizer)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [37]:
test_x.values()

dict_values([<tf.Tensor: shape=(2617, 100), dtype=int32, numpy=
array([[  101,  2054,  2024, ...,     0,     0,     0],
       [  101,  2079,  1045, ...,     0,     0,     0],
       [  101,  2065,  1045, ...,     0,     0,     0],
       ...,
       [  101,  2339,  3475, ...,     0,     0,     0],
       [  101,  2054,  9598, ...,     0,     0,     0],
       [  101,  2054, 12731, ...,     0,     0,     0]], dtype=int32)>])

In [42]:
test_pred=model.predict(test_x.values())

In [49]:
test_pred["category"]

array([[-0.8387513 ,  0.2761706 ,  0.9110924 , ...,  0.39926457,
        -0.39543995, -0.38857988],
       [-0.7976462 ,  0.46443275,  0.8564301 , ...,  0.33517578,
        -0.3429824 , -0.2999277 ],
       [-0.68713117,  0.322859  ,  0.74229604, ...,  0.3316057 ,
        -0.2453018 , -0.43145543],
       ...,
       [ 0.42537126, -0.0377568 , -0.62311673, ..., -0.07145664,
         0.49977908,  0.07321449],
       [-0.7243861 ,  0.20455024,  0.90237623, ...,  0.1975209 ,
        -0.3937879 , -0.30750853],
       [-0.71479726,  0.15032887,  0.9032665 , ...,  0.25571755,
        -0.39795306, -0.3019337 ]], dtype=float32)

In [50]:
import numpy as np
prediction = np.where(test_pred["category"]>0.5, 1,0)

In [51]:
prediction

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [ ]:
import tensorflow
tensorflow.keras.models.save_model(
    model,
   '/model_path/',
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None,
    save_traces=True,
)

INFO:tensorflow:Assets written to: /model_path/assets


INFO:tensorflow:Assets written to: /model_path/assets


In [28]:
#######################################
### ----- Evaluate the model ------ ###
# Ready test data
test_y_category =Y_test
test_x =batch_encode(X_test,tokenizer)
# Run evaluation
model_eval = model.evaluate(
    x={'input_ids': test_x['input_ids']},
    y={'category': test_y_category}
)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


82/82 [==============================] - 32s 386ms/step - loss: 3.9723 - accuracy: 0.0695
